# Sudoku (Human Agent)
Enter numbers 1-9 into the empty cells, ensuring each row, column, and 3x3 box only contains a single 

In [188]:
from IPython.display import display, HTML, Javascript, clear_output
import ipywidgets as widgets
import json

# Sudoku grid
sudoku_grid = [
    0, 6, 2, 4, 0, 5, 0, 0, 8,
    0, 0, 5, 0, 0, 0, 4, 2, 0,
    3, 0, 4, 9, 0, 0, 5, 6, 0,
    0, 0, 0, 6, 0, 2, 9, 8, 4,
    0, 2, 7, 0, 4, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 1, 0, 0, 0,
    4, 9, 6, 1, 5, 7, 8, 0, 0,
    2, 0, 8, 0, 0, 0, 7, 0, 5,
    7, 5, 0, 0, 0, 4, 0, 9, 6,
]

# Correct answers, rename to sudoku_grid to test
answer_grid = [
    9, 6, 2, 4, 1, 5, 3, 7, 8,
    1, 8, 5, 7, 6, 3, 4, 2, 9,
    3, 7, 4, 9, 2, 8, 5, 6, 1,
    5, 3, 1, 6, 7, 2, 9, 8, 4,
    8, 2, 7, 5, 4, 9, 6, 1, 3,
    6, 4, 9, 8, 3, 1, 2, 5, 7,
    4, 9, 6, 1, 5, 7, 8, 3, 2,
    2, 1, 8, 3, 9, 6, 7, 4, 5,
    7, 5, 3, 2, 8, 4, 1, 9, 6,
]

# Build HTML board
html = '''
<style>
.sudoku-table input {
    width: 30px;
    height: 30px;
    text-align: center;
    font-size: 16px;
    border: 1px solid lightgray;
}
.sudoku-table td {
    padding: 0;
}
.sudoku-table {
    border-collapse: collapse;
}
.sudoku-table td:nth-child(3),
.sudoku-table td:nth-child(6) {
    border-right: 2px solid black;
}
.sudoku-table tr:nth-child(3) td,
.sudoku-table tr:nth-child(6) td {
    border-bottom: 2px solid black;
}
.sudoku-table td:first-child {
    border-left: 2px solid black;
}
.sudoku-table tr:first-child td {
    border-top: 2px solid black;
}
.sudoku-table tr:last-child td {
    border-bottom: 2px solid black;
}
.sudoku-table td:last-child {
    border-right: 2px solid black;
}
</style>
<table class="sudoku-table">
'''

for r in range(9):
    html += '<tr>'
    for c in range(9):
        val = sudoku_grid[r * 9 + c]
        value_attr = f'value="{val}" disabled' if val != 0 else ''
        html += f'<td><input id="cell-{r}-{c}" type="text" maxlength="1" {value_attr}></td>'
    html += '</tr>'
html += '</table>'

display(HTML(html))

# Hidden Textarea for grid sync
data_holder = widgets.Textarea(value="[]", layout={'display': 'none'})
check_button = widgets.Button(description="Check Sudoku")
output = widgets.Output()
shared = {'answers': sudoku_grid}

def handle_data_holder_change(change):
    with output:
        try:
            shared['answers'] = [int(x) for row in json.loads(change['new']) for x in row]
        except Exception as e:
            print("Failed to parse grid:", e)

data_holder.observe(handle_data_holder_change, names='value')

def is_valid_sudoku(grid):
    # Validate rows and columns.
    for i in range(9):
        row = set()
        column = set()
        for j in range(9):
            # Validate rows
            row_cell = grid[i * 9 + j]
            if row_cell in row or row_cell == 0:
                return False
            row.add(row_cell) if row_cell != 0 else None

            # Validate columns
            column_cell = grid[j * 9 + i]
            if column_cell in column or column_cell == 0:
                return False
            column.add(column_cell) if column_cell != 0 else None

    # Validate 3x3 squares
    for block_i in range(3):
        for block_j in range(3):
            square = set()
            for i in range(3):
                for j in range(3):
                    index = (block_i * 3 + i) * 9 + (block_j * 3 + j)
                    if grid[index] in square or grid[index] == 0:
                        return False
                    square.add(
                        grid[index]) if grid[index] != 0 else None

    return True

def on_check_clicked(b):
    with output:
        clear_output()
        try:
            grid = shared['answers']
            if not (isinstance(grid, list) and len(grid) == 81):
                print("Grid format is invalid")
                return
            if is_valid_sudoku(grid):
                print("This Sudoku is valid")
            else:
                print("This Sudoku is invalid")
        except Exception as e:
            print("Error parsing grid:", e)

check_button.on_click(on_check_clicked)
display(data_holder, check_button, output)

display(Javascript("""
(() => {
    const button = [...document.querySelectorAll('button')].find(b => b.textContent.includes("Check Sudoku"));
    if (!button) return;

    button.addEventListener("click", () => {
        const grid = [];
        for (let r = 0; r < 9; r++) {
            const row = [];
            for (let c = 0; c < 9; c++) {
                const el = document.getElementById(`cell-${r}-${c}`);
                let val = el?.value.trim();
                let num = parseInt(val);
                if (isNaN(num)) num = 0;
                row.push(num);
            }
            grid.push(row);
        }

        const textArea = [...document.querySelectorAll('.widget-textarea textarea')][0];
        if (textArea) {
            textArea.value = JSON.stringify(grid);
            textArea.dispatchEvent(new Event("change", { bubbles: true }));
        } else {
            console.error("Couldn't find textarea to sync data.");
        }
    }, { once: false });
})();
"""))


,,,,,,,,
,,,,,,,,
,,,,,,,,
,,,,,,,,
,,,,,,,,
,,,,,,,,
,,,,,,,,
,,,,,,,,
,,,,,,,,


Textarea(value='[]', layout=Layout(display='none'))

Button(description='Check Sudoku', style=ButtonStyle())

Output()

<IPython.core.display.Javascript object>